<a href="https://colab.research.google.com/github/Giffy/AI_fast.ai/blob/master/Machine%20Learning/lesson3_grocery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Important: This notebook will only work with fastai-0.7.x. Do not try to run any fastai-1.x code from this path in the repository because it will load fastai-0.7.x**

# Intro to Random Forests - Favorita Grocery Sales Forecasting 

Notebook based in kaggle competition [Can you accurately predict sales for a large grocery chain?](https://www.kaggle.com/c/favorita-grocery-sales-forecasting)

Corporación Favorita has challenged the Kaggle community to build a model that more accurately forecasts product sales.

They’re excited to see how machine learning could better ensure they please customers by having just enough of the right products at the right time.

In this competition, you will be predicting the unit sales for thousands of items sold at different Favorita stores located in Ecuador. The training data includes dates, store and item information, whether that item was being promoted, as well as the unit sales. Additional files include supplementary information that may be useful in building your models.

##File Descriptions and Data Field Information

**train.csv**

* Training data, which includes the target *unit_sales by date, store_nbr*, and *item_nbr* and a unique *id* to label rows.
* The target unit_sales can be integer (e.g., a bag of chips) or float (e.g., 1.5 kg of cheese).
* Negative values of unit_sales represent returns of that particular item.
* The onpromotion column tells whether that item_nbr was on promotion for a specified date and store_nbr.
* Approximately 16% of the onpromotion values in this file are NaN.
* **NOTE:** The training data does not include rows for items that had zero unit_sales for a store/date combination. There is no information as to whether or not the item was in stock for the store on the date, and teams will need to decide the best way to handle that situation. Also, there are a small number of items seen in the training data that aren't seen in the test data.


**test.csv**

* Test data, with the date, store_nbr, item_nbr combinations that are to be predicted, along with the onpromotion information.
* **NOTE:** The test data has a small number of items that are not contained in the training data. Part of the exercise will be to predict a new item sales based on similar products..
* The public / private leaderboard split is based on time. All items in the public split are also included in the private split.


# Google Colab setup
Installs fast.ai 0.7.0 and the required libraries to run the notebook.

Also downloads the required datasets.

Train.csv dataset has been reduced in order to store the data sample in Github. 

Original data file can be downloaded in [kaggle](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data) (size of compressed file:  474Mb) 

In [0]:
print (" Installing FastAI libraries ... (takes 2 min)")
!pip install fastai==0.7.0 > /dev/null
!pip install scikit-learn==0.20 > null
print ("\n Clonning FastAI repository locally ...")
!git clone https://github.com/fastai/fastai.git fastai_ml
!ln -s fastai_ml/courses/ml1/fastai/ fastai

In [0]:
print ("\n Installing required libraries...")
!pip install --upgrade setuptools > /dev/null
!pip install feather > /dev/null
!pip install scikit-misc==0.1.0 > /dev/null
!pip install pdpbox==0.2.0 > /dev/null
!pip install treeinterpreter==0.2.2 > /dev/null
print ("\n Downloading datasets...")
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-train.7z
!7z x -y grocery-sales-train.7z > /dev/null
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/grocery-sales-test.tar.gz
!tar xvf grocery-sales-test.tar.gz > /dev/null
print ("\n Importing libraries")
import pandas as pd
import os
import numpy as np

# 1 Imports 

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.imports import *
from fastai.structured import *

from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

In [0]:
PATH = 'grocery-sales/'
!ls {PATH}

# 2 Read data

In [0]:
types = {'id': 'int64',
        'item_nbr': 'int32',
        'store_nbr': 'int8',
        'unit_sales': 'float32',
        'onpromotion': 'object'}

In [0]:
%%time
df_all = pd.read_csv(f'{PATH}train_basic.csv', parse_dates=['date'], dtype=types, 
                     infer_datetime_format=True)#, skiprows=range(1,100000000))

In [0]:
df_all.onpromotion.fillna(False, inplace=True)
df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_all.onpromotion = df_all.onpromotion.astype(bool)

os.makedirs('tmp', exist_ok=True)
%time df_all.to_feather('tmp/raw_groceries')

In [0]:
df_all.drop('Unnamed: 0', axis=1, inplace=True)
%time df_all.describe(include='all')

In [0]:
df_test = pd.read_csv(f'{PATH}test.csv',parse_dates=['date'], dtype=types, infer_datetime_format=True)

df_test.onpromotion.fillna(False, inplace=True)
df_test.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
df_test.onpromotion = df_all.onpromotion.astype(bool)
df_test.describe(include='all')

In [0]:
df_all.tail()

In [0]:
df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))

In [0]:
%time add_datepart(df_all, 'date')

In [0]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [0]:
n_valid = len(df_test)
n_trn = len(df_all) - n_valid
train, valid = split_vals(df_all, n_trn)
train.shape, valid.shape

In [0]:
# train_cats(raw_train)
# apply_cats(raw_valid, raw_train)

In [0]:
%%time
trn, y, nas = proc_df(train, 'unit_sales')
val, y_val, nas = proc_df(valid, 'unit_sales')

# 3 Models

In [0]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(x), y), rmse(m.predict(val), y_val),
          m.score(x, y), m.score(val, y_val)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [0]:
set_rf_samples(1000000)

In [0]:
%time x = np.array(trn, dtype=np.float32)

In [0]:
# set n_jobs=-1 to use all CPU cores available
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=100, n_jobs=-1) 
%time m.fit(x, y)

In [0]:
print_score(m)

In [0]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=10, n_jobs=-1)
%time m.fit(x, y)

In [0]:
print_score(m)

In [0]:
m = RandomForestRegressor(n_estimators=20, min_samples_leaf=3, n_jobs=-1)
%time m.fit(x, y)

In [0]:
%time print_score(m)